# Simulating Deletions

In [1]:
import pandas
from time import time

import cobra.test
from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

cobra_model = cobra.test.create_test_model("textbook")
ecoli_model = cobra.test.create_test_model("ecoli")

## Knocking out single genes and reactions

A commonly asked question when analyzing metabolic models is what will happen if a certain reaction was not allowed to have any flux at all. This can tested using cobrapy by

In [2]:
print('complete model: ', cobra_model.optimize())
with cobra_model:
    cobra_model.reactions.PFK.knock_out()
    print('pfk knocked out: ', cobra_model.optimize())

complete model:  <Solution 0.874 at 0x1118cc898>
pfk knocked out:  <Solution 0.704 at 0x1118cc5c0>


For evaluating genetic manipulation strategies, it is more interesting to examine what happens if given genes are knocked out as doing so can affect no reactions in case of redundancy, or more reactions if gene when is participating in more than one reaction.

In [3]:
print('complete model: ', cobra_model.optimize())
with cobra_model:
    cobra_model.genes.b1723.knock_out()
    print('pfkA knocked out: ', cobra_model.optimize())
    cobra_model.genes.b3916.knock_out()
    print('pfkB knocked out: ', cobra_model.optimize())

complete model:  <Solution 0.874 at 0x1108b81d0>
pfkA knocked out:  <Solution 0.874 at 0x1108b80b8>
pfkB knocked out:  <Solution 0.704 at 0x1108b8128>


## Single Deletions

Perform all single gene deletions on a model

In [4]:
deletion_results = single_gene_deletion(cobra_model)

These can also be done for only a subset of genes

In [5]:
single_gene_deletion(cobra_model, cobra_model.genes[:20])

flux   status
b0116  0.782351  optimal
b0118  0.873922  optimal
b0351  0.873922  optimal
b0356  0.873922  optimal
b0474  0.873922  optimal
b0726  0.858307  optimal
b0727  0.858307  optimal
b1241  0.873922  optimal
b1276  0.873922  optimal
b1478  0.873922  optimal
b1849  0.873922  optimal
b2296  0.873922  optimal
b2587  0.873922  optimal
b3115  0.873922  optimal
b3732  0.374230  optimal
b3733  0.374230  optimal
b3734  0.374230  optimal
b3735  0.374230  optimal
b3736  0.374230  optimal
s0001  0.211141  optimal

This can also be done for reactions

In [6]:
single_reaction_deletion(cobra_model, cobra_model.reactions[:20])

flux   status
ACALD               8.739215e-01  optimal
ACALDt              8.739215e-01  optimal
ACKr                8.739215e-01  optimal
ACONTa             -5.039994e-13  optimal
ACONTb             -1.477823e-12  optimal
ACt2r               8.739215e-01  optimal
ADK1                8.739215e-01  optimal
AKGDH               8.583074e-01  optimal
AKGt2r              8.739215e-01  optimal
ALCD2x              8.739215e-01  optimal
ATPM                9.166475e-01  optimal
ATPS4r              3.742299e-01  optimal
Biomass_Ecoli_core  0.000000e+00  optimal
CO2t                4.616696e-01  optimal
CS                  1.129472e-12  optimal
CYTBD               2.116629e-01  optimal
D_LACt2             8.739215e-01  optimal
ENO                 1.161773e-14  optimal
ETOHt2r             8.739215e-01  optimal
EX_ac_e             8.739215e-01  optimal

## Double Deletions

Double deletions run in a similar way. Passing in `return_frame=True` will cause them to format the results as a `pandas.DataFrame`.

In [7]:
double_gene_deletion(
    cobra_model, cobra_model.genes[-5:], return_frame=True).round(4)

b2464   b0008   b2935   b2465  b3919
b2464  0.8739  0.8648  0.8739  0.8739  0.704
b0008  0.8648  0.8739  0.8739  0.8739  0.704
b2935  0.8739  0.8739  0.8739  0.0000  0.704
b2465  0.8739  0.8739  0.0000  0.8739  0.704
b3919  0.7040  0.7040  0.7040  0.7040  0.704

By default, the double deletion function will automatically use multiprocessing, splitting the task over up to 4 cores if they are available. The number of cores can be manually specified as well. Setting use of a single core will disable use of the multiprocessing library, which often aids debugging.

In [8]:
start = time()  # start timer()
double_gene_deletion(
    ecoli_model, ecoli_model.genes[:300], number_of_processes=2)
t1 = time() - start
print("Double gene deletions for 200 genes completed in "
      "%.2f sec with 2 cores" % t1)

start = time()  # start timer()
double_gene_deletion(
    ecoli_model, ecoli_model.genes[:300], number_of_processes=1)
t2 = time() - start
print("Double gene deletions for 200 genes completed in "
      "%.2f sec with 1 core" % t2)

print("Speedup of %.2fx" % (t2 / t1))

Double gene deletions for 200 genes completed in 33.26 sec with 2 cores
Double gene deletions for 200 genes completed in 45.38 sec with 1 core
Speedup of 1.36x


Double deletions can also be run for reactions.

In [9]:
double_reaction_deletion(
    cobra_model, cobra_model.reactions[2:7], return_frame=True).round(4)

ACKr  ACONTa  ACONTb   ACt2r    ADK1
ACKr    0.8739     0.0     0.0  0.8739  0.8739
ACONTa  0.0000     0.0     0.0  0.0000  0.0000
ACONTb  0.0000     0.0     0.0  0.0000 -0.0000
ACt2r   0.8739     0.0     0.0  0.8739  0.8739
ADK1    0.8739     0.0    -0.0  0.8739  0.8739